# Part(4): Обучение представлений

## 1. Данные

Будем работать с данными из домашнего соревнования

In [ ]:
import pandas as pd
from etna.datasets import TSDataset

import warnings

warnings.filterwarnings("ignore")

HORIZON = 365

In [ ]:
df = pd.read_parquet("data/train.parquet", columns=["segment", "timestamp", "target"])
ts = TSDataset(df=df, freq="D")
ts.plot(n_segments=4)

## 2. Эксперимент(1): Смоделируем сезонность

Попробуем смоделировать сезонность с помощью признаков из временной метки с помощью глобальной модели

In [ ]:
from etna.pipeline import Pipeline
from etna.transforms import DateFlagsTransform
from etna.models import CatBoostMultiSegmentModel
from etna.metrics import SMAPE

### Создаем пайплайн
1. Модель CatBoostMultiSegmentModel
2. Признаки из временной метки для моделирования недельной/годовой сезонности

In [ ]:
pipeline = Pipeline(
    ...  # <your code here>
)

### Запускаем кросс-валидацию

Запустим кросс-валидацию на 1 фолде и оценим метрику соревнования SMAPE

In [ ]:
# <your code here>

### Нарисуем картинку

Качество получилось так себе, если нарисовать картинку то проблема станет очевидна

In [ ]:
from etna.analysis import plot_backtest

In [ ]:
# <your code here>

## 3. Реализация TSFreshSegmentEncoder

Попробуем исправить проблему, добавив в модель признаковое представление каждого сегмента в датасете.

В качестве признакового представления будем использовать признаки из tsfresh

Реализуем кастомный трансформ, который будет выделять нужные признаки([туториал](https://github.com/etna-team/etna/blob/master/examples/301-custom_transform_and_model.ipynb))

In [ ]:
from etna.transforms.base import IrreversibleTransform

from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters

from typing import Optional, List

In [ ]:
class TSFreshSegmentEncoder(IrreversibleTransform):
    def __init__(self, in_column: str, default_fc_parameters: Optional[dict] = None, n_jobs: int = 1, **tsfresh_kwargs):
        """
        Create instance of TSFreshFeatrureExtractionTransform.

        Parameters
        ----------
        in_column:
            name of processed column
        default_fc_parameters:
            mapping from feature calculator names to parameters
        n_jobs:
            the number of processes to use for parallelization. If zero, no parallelization is used.
        tsfresh_kwargs:
            other arguments of extract_features method
        """
        super().__init__(required_features=[in_column])
        self.in_column = in_column
        self.default_fc_parameters = default_fc_parameters
        self.n_jobs = n_jobs
        self.tsfresh_kwargs = tsfresh_kwargs

        self.output_columns: Optional[list[str]] = None
        self.in_column_regressor: Optional[bool] = None

    def fit(self, ts: TSDataset) -> "TSFreshFeatrureExtractionTransform":
        """Fit the transform."""
        # В зависимости от того, является ли входная колонка регрессором, будем определять будут ли регрессорами выходные
        self.in_column_regressor = self.in_column in ts.regressors
        super().fit(ts)
        return self

    def _fit(self, df: pd.DataFrame) -> "TSFreshFeatrureExtractionTransform":
        """
        Calculate the indices that need to be changed.

        Returns
        -------
        self
        """
        df = TSDataset.to_flatten(df)  # Dataframe с колонками (timestamp, segment, self.in_column)

        # В этом месте хотим понять какие у нас будут выходные колонки
        # Для этого проделаем такую же процедуру как в методе _transform, только на маленьком подмножестве точек датасета
        # Лучше взять 1 сегмент и его последние 10 точек

        df_featured = ...  # <your code here>

        self.output_columns = df_featured.columns
        return self

    def _transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract tsfresh features

        Parameters
        ----------
        df:
            DataFrame to transform

        Returns
        -------
        transformed series
        """

        df = TSDataset.to_flatten(df)  # Dataframe с колонками (timestamp, segment, self.in_column)

        # Предлагается выделить фичи для каждого сегмента с помощью extract_features
        # Для каждого сегмента должны получить общее признаковое представление и продублировать его во все точки временного ряда

        # Важно: в in_column могут присутствовать просуски в начале ряда например из-за того что ряды в датасете разной длинны
        # Перед подсчетом фичей необходимо избавиться от этих пропусков иначе все упадет

        df_featured = (
            ...
        )  # <your code here> -- на выходе Dataframe с колонками (timestamp, segment, self.in_column, feature_1, ...)

        df_featured = TSDataset.to_dataset(df_featured)  # перевод в wide формат

        return df_featured

    def get_regressors_info(self) -> List[str]:
        """Return the list with regressors created by the transform.

        Returns
        -------
        :
            List with regressors created by the transform.
        """
        if self.in_column_regressor:
            return self.output_columns
        return []

### Протестируем реализацию на 2 сегментах

In [ ]:
df = pd.read_parquet("data/train.parquet", columns=["segment", "timestamp", "target"])
df = df[df["segment"].isin(ts.segments[:2])]
ts = TSDataset(df=df, freq="D")
ts.head()

In [ ]:
tsfresh_transform = TSFreshSegmentEncoder(in_column="target", default_fc_parameters=MinimalFCParameters(), n_jobs=1)
ts.fit_transform([tsfresh_transform])
ts.tail()

## 4. Эксперимент(2): Смоделируем сезонность + добавим признаки сегмента

Теперь помимо сезонных признаков добавим в пайплайн признаковое описание сегмента

Важный момент, что для подсчета признаков мы можем использовать только данные начиная с T-Horizon так как для будущего неизвестны последие Horizon точек ряда. Для этого проделаем следующий трюк
1. Посчитаем признаки от Lag(Horizon)
2. Дропнем Lag(Horizon) -- хотя это в целом тоже разумный признак

In [ ]:
from etna.transforms import LagTransform, FilterFeaturesTransform

In [ ]:
df = pd.read_parquet("data/train.parquet", columns=["segment", "timestamp", "target"])
ts = TSDataset(df=df, freq="D")

In [ ]:
pipeline = Pipeline(
    ...  # <your code here>
)

### Запускаем кросс-валидацию
Запустим кросс-валидацию на 1 фолде и оценим метрику соревнования SMAPE

In [ ]:
# <your code here>

### Нарисуем картинку

Получилось сильно лучше, однако результат естественно не идеален

In [ ]:
# <your code here>